**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

Read the RDF file

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")
print(g.serialize(format="ttl").decode("UTF-8"))

@prefix ns: <http://somewhere#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix vcard-rdf: <http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ns:Person a rdfs:Class .

ns:PhDstudent a rdfs:Class ;
    rdfs:subClassOf ns:Researcher .

ns:Researcher a rdfs:Class ;
    rdfs:subClassOf ns:Person .

ns:University a rdfs:Class .

ns:JaneSmith a ns:Researcher ;
    vcard-rdf:FN "jane Smith"^^xsd:string ;
    vcard-rdf:Family "Smith"^^xsd:string ;
    vcard-rdf:Given "Jane"^^xsd:string .

ns:JohnSmith a ns:Person ;
    vcard-rdf:FN "John Smith"^^xsd:string ;
    vcard-rdf:Family "Smith"^^xsd:string ;
    vcard-rdf:Given "John"^^xsd:string .

ns:SaraJones a ns:Person ;
    vcard-rdf:FN "Sara Jones"^^xsd:string ;
    vcard-rdf:Family "Jones"^^xsd:string ;
    vcard-rdf:Given "Sara"^^xsd:string .

ns:UPM a ns:University .

vcard-rdf:FN a rdfs:Datatype ;
    rdfs:range xsd:string .

vcard-rdf:Family a rdfs:Property ;
   

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [12]:
# RDFLib
somewhere=Namespace("http://somewhere#")
for a, b, c in g.triples((None, RDFS.subClassOf, somewhere.Person)):
   print(a)

http://somewhere#Researcher


In [23]:
# SPARQL
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdfs:subClassOf <http://somewhere#Person>. 
  }
  '''
)
print(q1)
# for r in g.query(q1):
#   print(r)

**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [11]:
# RDFLib
for a, b, c in g.triples((None, RDF.type, somewhere.Person)):
  print(a)

http://somewhere#JohnSmith
http://somewhere#SaraJones


In [22]:
# SPARQL
q1 = prepareQuery('''
  SELECT 
    ?Subject
  WHERE { 
    ?Subject rdf:type <http://somewhere#Person>. 
  }
  '''
)

for r in g.query(q1):
  print(r)


(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [32]:
# RDFLib

for a, b, c in g.triples((None, RDF.type, somewhere.Person)):
  for a, b, c in g.triples((a, None, None)):
    print(a, b)

http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN


In [26]:
# SPARQL
q1 = prepareQuery('''
  SELECT 
    ?Subject ?Property
  WHERE { 
    ?Subject rdf:type <http://somewhere#Person>.
    ?Subject ?Property ?value 
  }
  '''
)

for r in g.query(q1):
  print(r)


(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere